<a href="https://colab.research.google.com/github/rz-pb/Bioinformatics-Codes/blob/main/Local_Pairwise_Sequence_Alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Local Pairwise Sequence Alignment

In [1]:
import numpy as np
np.set_printoptions(linewidth=200)

In [2]:
V = "TACGGGTAT"
W = "GGACGTACG"

# Gap Penalty
d = 1

# Scoring Matrix
Scoring_Matrix = np.array([[ 1,    -1,      -1,     -1],
                           [-1,     1,      -1,     -1],
                           [-1,    -1,       1,     -1],
                           [-1,    -1,      -1,      1]])
 




# In case you want to check another example, call function LPSA_DP_BU with following data:

X = "CTCTGCCTCTG"
Y = "CACTCCTGATG"

# Gap Penalty
Another_d = 2

# Scoring Matrix
Another_Scoring_Matrix = np.array([[ 1,    -1,      -1,      0],
                                   [-1,     1,      -1,     -1],
                                   [-1,    -1,       1,     -1],
                                   [ 0,    -1,      -1,      1]])

In [3]:
def Score(v,u,Score_Matrix) :
  
  Residue_Index = {'A' : 0 , 'C' : 1 , 'G' : 2 , 'T' : 3}
  return Score_Matrix[Residue_Index[v],Residue_Index[u]]

In [4]:
def LPSA_DP_tables(X,Y) :
  
  LPSA_BU_table_temp = np.full((len(Y)+1,len(X)+1), -1)

  LPSA_BU_operations_table_temp = (len(Y)+1)*(len(X)+1)*["E"]
  LPSA_BU_operations_table_temp = np.array(LPSA_BU_operations_table_temp , dtype=str)
  LPSA_BU_operations_table_temp = np.reshape(LPSA_BU_operations_table_temp,(len(Y)+1,len(X)+1))

  return (LPSA_BU_table_temp , LPSA_BU_operations_table_temp) 

In [5]:
def LPSA_DP_BU(X,Y,S,d) :


  LPSA_BU_table , LPSA_BU_operations_table = LPSA_DP_tables(X,Y)
  
  

  for i in range(0,len(Y)+1) :
    for j in range(0,len(X)+1) :
      
      if i == 0 and j == 0 :
        LPSA_BU_table[i,j] = 0
        LPSA_BU_operations_table[i,j] = "N"

      if i == 0 and j != 0 :
        LPSA_BU_table[i,j] = max(0 , -1 * j * d)
        
        if LPSA_BU_table[i,j] == -1 * j * d :
          LPSA_BU_operations_table[i,j] = "D"   # DELETE

        if LPSA_BU_table[i,j] == 0 :
          LPSA_BU_operations_table[i,j] = "N" # NONE

      
      if j == 0 and i != 0 :
        LPSA_BU_table[i,j] = max(0 , -1 * i * d)
        
        if LPSA_BU_table[i,j] == -1 * i * d :
          LPSA_BU_operations_table[i,j] = "I"   # INSERT

        if LPSA_BU_table[i,j] == 0 :
          LPSA_BU_operations_table[i,j] = "N" # NONE  


      
      if i > 0 and j > 0 :


          LPSA_BU_table[i,j] = max( 0 , LPSA_BU_table[i-1,j-1] + Score(X[j-1],Y[i-1],S) , LPSA_BU_table[i,j-1] - d , LPSA_BU_table[i-1,j] - d )
          
          if LPSA_BU_table[i,j] == LPSA_BU_table[i-1,j-1] + Score(X[j-1],Y[i-1],S) :
            LPSA_BU_operations_table[i,j] = "S" # SUBSTITUTION

          if LPSA_BU_table[i,j] == LPSA_BU_table[i,j-1] - d :
            LPSA_BU_operations_table[i,j] = "D" # DELETE

          if LPSA_BU_table[i,j] == LPSA_BU_table[i-1,j] - d :
            LPSA_BU_operations_table[i,j] = "I" # INSERT

          if LPSA_BU_table[i,j] == 0 :
            LPSA_BU_operations_table[i,j] = "N" # NONE

            

  return (LPSA_BU_table , LPSA_BU_operations_table , LPSA_BU_table.max())

In [6]:
local_optimal_value_table , local_optimal_solution_table ,local_optimal_value = LPSA_DP_BU(V,W,Scoring_Matrix,d)

In [7]:
local_optimal_value_table

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 2, 2, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 1, 1, 2, 1],
       [0, 0, 0, 2, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 3, 2, 1, 0, 0, 0],
       [0, 1, 0, 0, 2, 2, 1, 2, 1, 1],
       [0, 0, 2, 1, 1, 1, 1, 1, 3, 2],
       [0, 0, 1, 3, 2, 1, 0, 0, 2, 2],
       [0, 0, 0, 2, 4, 3, 2, 1, 1, 1]])

In [8]:
local_optimal_solution_table

array([['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N'],
       ['N', 'N', 'N', 'N', 'S', 'S', 'S', 'N', 'N', 'N'],
       ['N', 'N', 'N', 'N', 'S', 'S', 'S', 'D', 'N', 'N'],
       ['N', 'N', 'S', 'N', 'N', 'I', 'I', 'S', 'S', 'D'],
       ['N', 'N', 'N', 'S', 'D', 'N', 'N', 'N', 'I', 'S'],
       ['N', 'N', 'N', 'I', 'S', 'D', 'D', 'N', 'N', 'N'],
       ['N', 'S', 'N', 'N', 'I', 'S', 'D', 'S', 'D', 'S'],
       ['N', 'N', 'S', 'D', 'I', 'I', 'S', 'I', 'S', 'D'],
       ['N', 'N', 'I', 'S', 'D', 'D', 'N', 'N', 'I', 'S'],
       ['N', 'N', 'N', 'I', 'S', 'D', 'D', 'D', 'I', 'I']], dtype='<U1')

In [9]:
local_optimal_value

4